In [1]:
import tensorflow as tf
from tensorflow.python.client import device_lib
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices()) 

print(tf.__version__)

from tensorflow import test
test.is_gpu_available()

['/device:CPU:0', '/device:XLA_CPU:0', '/device:XLA_GPU:0', '/device:GPU:0']
2.2.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True